In [4]:
import os

with open('api_key.txt', 'r') as f:
    key = f.read()

os.environ['GIGACHAT_API_ACCESS_KEY'] = key

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter 
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_paths = [
    'docs/test.json',
    'docs/raw_recomm_papers_1.txt',
    'docs/raw_recomm_papers_2.txt',
    'docs/raw_recomm_papers_3.txt'
]

# Load each file and store the results in a list
documents = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding='utf-8')  # Specify the encoding
    documents.extend(loader.load())


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
db

C:\Users\gunte\AppData\Local\Temp\ipykernel_10040\1814353829.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\gunte\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke("Ноутбук для учёбы")

retrieved_docs

[Document(metadata={'source': 'docs/raw_recomm_papers_2.txt'}, page_content='i5 8го поколения, сенсорный FullHD-экран, SSD на 256гб, батарея в состоянии "муха не сидела" и 8гб оперативки, которую я нарастил до 16гб.'),
 Document(metadata={'source': 'docs/test.json'}, page_content='"раскладка_клавиатуры": "английская/русская",\n      "цена": "20,000 ₽"\n    }\n  ]'),
 Document(metadata={'source': 'docs/raw_recomm_papers_2.txt'}, page_content='Подобрать ноутбук можно и\xa0за\xa0меньшие деньги: бывают модели дешевле 25\xa0000\xa0₽, но\xa0использовать их не\xa0очень комфортно.«Глаза не\xa0устают»: как я\xa0выбирал ноутбук для\xa0школьника до\xa050\xa0000\xa0₽84826Разъемы, интерфейсы, важные мелочи. Ноутбук\xa0— это устройство «все в\xa0одном», поэтому при\xa0покупке стоит учитывать множество деталей:Стандартные USB-А пригодятся для\xa0флешек и\xa0простой периферии вроде мышек, клавиатур и\xa0принтеров.USB-C бывает разных типов, но\xa0обычно используется для\xa0зарядки, подключения продвину

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [8]:
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_ACCESS_KEY'],
    model="GigaChat",
    verify_ssl_certs=False,
    profanity_check=False,
)

prompt = ChatPromptTemplate.from_template(''' Ты - консультант в техническом интернет-магазине \
                                          Ответь на вопрос пользователя. \
Используй при этом информацию из контекста. Если в контексте нет \
информации для ответа, попроси пользователя уточнить необходимые детали.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)


C:\Users\gunte\AppData\Local\Temp\ipykernel_10040\4017348131.py:1: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(


In [9]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [10]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
retrieval_chain.invoke({'input': "Какой лучше ноутбук подойдёт для учёбы (школьник 9-го класса)?"})

{'input': 'Какой лучше ноутбук подойдёт для учёбы (школьник 9-го класса)?',
 'context': [Document(metadata={'source': 'docs/raw_recomm_papers_2.txt'}, page_content='i5 8го поколения, сенсорный FullHD-экран, SSD на 256гб, батарея в состоянии "муха не сидела" и 8гб оперативки, которую я нарастил до 16гб.'),
  Document(metadata={'source': 'docs/raw_recomm_papers_2.txt'}, page_content='Подобрать ноутбук можно и\xa0за\xa0меньшие деньги: бывают модели дешевле 25\xa0000\xa0₽, но\xa0использовать их не\xa0очень комфортно.«Глаза не\xa0устают»: как я\xa0выбирал ноутбук для\xa0школьника до\xa050\xa0000\xa0₽84826Разъемы, интерфейсы, важные мелочи. Ноутбук\xa0— это устройство «все в\xa0одном», поэтому при\xa0покупке стоит учитывать множество деталей:Стандартные USB-А пригодятся для\xa0флешек и\xa0простой периферии вроде мышек, клавиатур и\xa0принтеров.USB-C бывает разных типов, но\xa0обычно используется для\xa0зарядки, подключения продвинутых'),
  Document(metadata={'source': 'docs/raw_recomm_papers